# Libraries

In [1]:
import numpy as np

# Defined function

Пусть мы хотим обучить сеть классифицировать числа. Рассмотрим функцию $f(x)=x>0$. Для простоты рассматривать будем числа на отрезке $[-5, 5] \subset \mathbb{R}$. 

- $x\in[-5, 5]\cap\mathbb{R}$;
- $f(x) = x>0$;
- Функция ошибки $CrossEntropy(model) = -\sum_{i=1}^{100}(i>0)\log(model(i))$, где $model$ -- **однослойная нейронная сеть**; $model(i)$ -- **результат работы сети с входом i**;
- Обучать будем 100 эпох.

In [2]:
def f(x):
    return x>0

In [6]:
dif = 0
for i in range(-5, 6):
    dif += f(i)*2-1

print(dif)

-1


# Data

Для корректной работы сети, сперва нужно правильно сгруппировать данные, разделив их на 3 части:
- Тестовая выборка - эту выборку алгоритм не использует в обучении, на ней будут показаны только итоговые значения точности
- Тренировочная выборка - на этих данных сеть обучается:
    - Тренировочная - на этих данных алгоритм обучает сеть
    - Валидационная - на этих данных алгоритм после 1 итерации обучения оценивает показатели(затем может менять параметры сети для улучшения обучаемости)
   
Напишем функцию, производяющую деление выборки в соотношении fraction\*len для обучения и (1-fraction)\*len для тестирования.

**P.s.** Самое главное, обучать сеть на равномерно распределенной выборке, для этого будем делить на описанные выше соотнешения элементы каждого класса(всего их 2), чтобы получить проавильно разбитые данные.

In [3]:
def random_sample(X, y, fraction=0.8):
    size=X.shape[0]
    per_class = int(0.5*fraction*size)
    first_rand_idxs = np.random.choice(np.arange(size//2), per_class, replace=False)
    second_rand_idxs = np.random.choice(np.arange(size//2, size), per_class, replace=False)
    train_idxs = sorted([*first_rand_idxs, *second_rand_idxs])
    test_idxs = np.array([i for i in range(size) if i not in train_idxs])
    X_train, X_test = X[train_idxs], X[test_idxs]
    y_train, y_test = y[train_idxs], y[test_idxs]
    
    return X_train, X_test, y_train, y_test

In [4]:
X = np.linspace(-5, 5, num=100)
y = f(X)

In [5]:
X_train, X_test, y_train, y_test = random_sample(X, y)

In [6]:
print(X_train.shape)
print(X_test.shape)

(80,)
(20,)


## Validation & train

Аналогично, разделим тренировочную выборку на тренировочную и валидационную.

In [11]:
X_nn_train, X_nn_val, y_nn_train, y_nn_val = random_sample(X_train, y_train)

In [12]:
print(X_nn_train.shape)
print(X_nn_val.shape)

(64,)
(16,)


# Сама модель

Рассмотрим в качестве модели линейный классификатор(1 нейрон) с одной постоянной смещения(в нашем случае она равна 0).

$model(x) = x*w+b$

Затем к выходу будет применяться нелинейный слой активации -- **сигмоида**: $sigmoid(x) = \frac{1}{1+e^{-x}}$

In [7]:
def sigmoid(x, eps=1e-5):
    return 1/(1+np.exp(-x))

class Net:
    def __init__(self, w=0.5, b=1):
        self.w = w
        self.b = b
    
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x):
        return sigmoid(x*self.w+self.b)

In [8]:
net = Net(w=0.5, b=1)
print(net(np.array([1])))

[0.81757448]


# Функция ошибки

$L(x) = -\frac{1}{16}\sum_{i=1}^{16}(y\_val[i]*\log(model(X\_nn\_val[i]))+(1-y\_val[i])*\log(1-model(X\_nn\_val[i])))$

Реализуем функцию ошибки для одиночного элемента, т.е. 1 слагаемое суммы сверху. Учтем, что из-за больших и малых чисел могут быть переполнения, поэтому в этих местах заменим значение на специально выбранное минимальное допустимое $e^{-5}$. 

In [9]:
def BCELoss(x, y, eps=1e-5):
    new_x = np.where(x>eps, x, 10)
    ans = np.zeros(x.shape)
    ans[new_x>2] = 1
    new_x = np.where(new_x<1-eps, new_x, -10)
    ans[new_x>0] = -y[new_x>0]*np.log(new_x[new_x>0])-(1-y[new_x>0])*np.log(1-new_x[new_x>0])
    return ans

In [10]:
print(sigmoid(-100))

3.7200759760208356e-44


In [11]:
print(BCELoss(sigmoid(np.array([100])), np.array([1])))

[0.]


## Обратное распространение

Уточним действие сети:

1. Вход домножается на вес **w**: $r_1 = w\times x$;
2. К результату прибавляется константное смещение **b**: $l = w\times x + b$;
3. К полученным значениям применяется преобразование функцией-сигмоидой **r** = $sigmoid(l)$;
4. Ошибка работы оценивается через **BCELoss**, сравнивая **r** с **y** -- известной меткой.

Для обучения сети, как мы уже разбирали, мы должны идти от ошибки и постепенно обновлять веса от последнего слоя к первому.

1. Оцениваем прозводную ошибки на выходе, т.к. решаем градиентным методом: $\frac{\partial L}{\partial r} = -\frac{\partial \left(y*\log(r)+(1-y)*\log(1-r)\right)}{\partial r} = -\frac{y}{r} + \frac{1-y}{1-r}$. Для ее вычисления мы знаем все значения;
2. Теперь переходим к слою активации, т.е. к сигмоиде. Здесь нужно использовать преобразования: $\frac{\partial r}{\partial l}$, где **r** полагаем равным значению сигмоиды от **l**, $\frac{\partial r}{\partial l} = \frac{\partial \frac{1}{1+e^{-l}}}{\partial l} = \frac{e^l(e^l+1)-e^{2l}}{(1+e^{l})^2} = \frac{e^l}{(1+e^{l})^2} = \frac{e^{-l}}{(1+e^{-l})^2} = \frac{1}{1+e^{-l}}*\left(1-\frac{1}{1+e^{-l}}\right) = r(1-r)$. Здесь мы получили значение производной через значение самой функции, поэтому сразу же можем ее вычислить. Далее пользуемся тем фактом, что $\frac{\partial L}{\partial l} = \frac{\partial L}{\partial r} * \frac{\partial r}{\partial l}$;
3. Осталось, наконец получить правильные значения коррекции настраеваемых параметров **w**, **b**: $\frac{\partial l}{\partial w} = \frac{\partial (w\times x + b)}{\partial w} = x$; $\frac{\partial l}{\partial b} = \frac{\partial (w\times x + b)}{\partial b} = 1$. Поэтому для **w** $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial l} * \frac{\partial l}{\partial w} = \frac{\partial L}{\partial r} * r(1-r)*x$; А для **b**: 
$\frac{\partial L}{\partial b} = \frac{\partial L}{\partial l} * \frac{\partial l}{\partial b} = \frac{\partial L}{\partial r} * r(1-r)$

После того, как мы получили значения для производных каждого из весов, можем их менять, руководствуясь тем, о чем говорилось в теории(скорость обучения):

$w = w - \alpha*\frac{\partial L}{\partial w}$

$b = b - \alpha*\frac{\partial L}{\partial b}$

Теперь полученные функции занесем в метод обучения сети.

In [12]:
def backprop(net: Net, x, y, lr=0.01):
    r = net(x)
    #print(r)
    divider_1 = np.where(r>1e-5, r, 1e-5)
    divider_2 = np.where(1-r>1e-5, 1-r, 1e-5)
    output_grad = -y/divider_1+(1-y)/divider_2
    #print(output_grad)
    #print((lr*output_grad*r*(1-r)*x).mean())
    #print((lr*output_grad*r*(1-r)).mean())
    net.w -= (lr*output_grad*r*(1-r)*x).mean()
    net.b -= (lr*output_grad*r*(1-r)).mean()

# Сам процесс обучения

У нас есть данные, на которых мы обучаем модель **X_nn_train**, на которых сравниваем качество в процессе обучения **X_nn_val**, наконец, те, на которых мы получим итоговое значение ошибки, заодно и точности **X_test**

Есть сама модель **Net**, есть функция ошибки **BCELoss**, даже есть функция обновления весов для обучения **backprop**.

Оставим исходными параметрами те, что заданы, обучим модель за 1000 итераций и посмотрим на точность каждой 100-ой итерации.

In [15]:
net = Net(w=0.7, b=0.1)

#X_nn_train = X_nn_train.astype(np.int32)
#y_nn_train = y_nn_train.astype(np.int32)
#X_nn_val = X_nn_val.astype(np.int32)
#y_nn_val = y_nn_val.astype(np.int32)
X_train = X_train.astype(np.int32)
y_train = y_train.astype(np.int32)
X_test = X_test.astype(np.int32)
y_test = y_test.astype(np.int32)
X_nn_val = X_test
y_nn_val = y_test
X_nn_train = X_train
y_nn_train = y_train

print(f'Default quality: BCE={BCELoss(net(X_nn_val), y_nn_val).mean()}, Accuracy = {(net(X_nn_val).round()==y_nn_val).mean()*100}%')
for it in range(10):
    backprop(net, X_nn_train, y_nn_train, lr=0.1)
    if it%1 == 0:
        print(net.w)
        print(net.b)
        print(f'Epoch №{it} -- quality: BCE={BCELoss(net(X_nn_val), y_nn_val).mean()}, Accuracy = {(net(X_nn_val).round()==y_nn_val).mean()*100}%')

print(f'Result quality: BCE={BCELoss(net(X_nn_val), y_nn_val).mean()}, Accuracy = {(net(X_nn_val).round()==y_nn_val).mean()*100}%')
print(net.w)
print(net.b)
print(f'Test quality: BCE={BCELoss(net(X_test), y_test).mean()}, Accuracy = {(net(X_test).round()==y_test).mean()*100}%')

Default quality: BCE=0.39386912930938783, Accuracy = 75.0%
0.7267569161149006
0.09981342050429531
Epoch №0 -- quality: BCE=0.3884007833156735, Accuracy = 75.0%
0.752084625318373
0.09969512281747835
Epoch №1 -- quality: BCE=0.38346623739566066, Accuracy = 75.0%
0.7761399213102382
0.0996380013210017
Epoch №2 -- quality: BCE=0.37898619521843246, Accuracy = 75.0%
0.7990550673077262
0.09963594767842882
Epoch №3 -- quality: BCE=0.37489668967777917, Accuracy = 75.0%
0.8209426198002476
0.0996836742335457
Epoch №4 -- quality: BCE=0.37114552197370576, Accuracy = 75.0%
0.8418991344210338
0.09977657421415606
Epoch №5 -- quality: BCE=0.3676896502619093, Accuracy = 75.0%
0.8620080475197052
0.09991061012725622
Epoch №6 -- quality: BCE=0.36449324560368046, Accuracy = 75.0%
0.8813419419175387
0.10008222391783388
Epoch №7 -- quality: BCE=0.3615262245971588, Accuracy = 75.0%
0.8999643469392531
0.10028826406656934
Epoch №8 -- quality: BCE=0.35876312769719043, Accuracy = 75.0%
0.9179311821918247
0.10052592